In [ ]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
# plt.style.use("ggplot")
%matplotlib inline

# from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from tensorflow.keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
im_width = 128
im_height = 128
border = 5
path_data = '../../../Desktop/training/img/'
path_label = '../../../Desktop/training/label/'
train_num = 2266

In [ ]:
# Get and resize train images and masks

x = np.zeros((train_num, im_height, im_width, 1), dtype=np.float32)
y = np.zeros((train_num, im_height, im_width, 1), dtype=np.float32)

index = 0
for fname in os.listdir(path_data):
    img = load_img(path_data + fname, grayscale=True)
    x_img = img_to_array(img)
    x_img = resize(x_img, (128, 128, 1), mode='constant', preserve_range=True)

    img = load_img(path_label + fname, grayscale=True)
    y_img = img_to_array(img)
    y_img = resize(y_img, (128, 128, 1), mode='constant', preserve_range=True)
    
    
    # Save images
    x[index, ..., 0] = x_img.squeeze() / 255
    y[index] = y_img / 255
    
    index += 1

In [ ]:
im_width = 128
im_height = 128
border = 5
path_data = '../../../Desktop/new/img/'
path_label = '../../../Desktop/new/label/'
predict_len = 172

x_new = np.zeros((predict_len, im_height, im_width, 1), dtype=np.float32)
y_new = np.zeros((predict_len, im_height, im_width, 1), dtype=np.float32)

index = 0
for fname in os.listdir(path_data):
    img = load_img(path_data + fname, grayscale=True)
    x_img = img_to_array(img)
    x_img = resize(x_img, (128, 128, 1), mode='constant', preserve_range=True)

    img = load_img(path_label + fname, grayscale=True)
    y_img = img_to_array(img)
    y_img = resize(y_img, (128, 128, 1), mode='constant', preserve_range=True)
    
    
    # Save images
    x_new[index, ..., 0] = x_img.squeeze() / 255
    y_new[index] = y_img / 255
    
    index += 1

In [ ]:
# Split train and valid
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.15, random_state=2018)

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

In [ ]:
def get_unet(input_img, n_filters=16, dropout=0.5, batchnorm=True):
    # contracting path
    c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [ ]:
input_img = Input((im_height, im_width, 1), name='img')
model = get_unet(input_img, n_filters=16, dropout=0.05, batchnorm=True)

model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model-tgs-salt.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [ ]:
# results = model.fit(x_train, y_train, batch_size=32, epochs=100, callbacks=callbacks, validation_split=0.3)

results = model.fit(x_train, y_train, batch_size=32, epochs=100, callbacks=callbacks, 
                    validation_data=(x_valid,y_valid))

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(results.history["loss"], label="loss")
plt.plot(results.history["val_loss"], label="val_loss")
plt.plot( np.argmin(results.history["val_loss"]), np.min(results.history["val_loss"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

In [ ]:
# Load best model
model.load_weights('model-tgs-salt.h5')

In [ ]:
# Evaluate on validation set (this must be equals to the best log_loss)
# model.evaluate(X_valid, y_valid, verbose=1)
model.evaluate(x_new, y_new, verbose=1)

In [ ]:
# Predict on train, val and test
# preds_train = model.predict(x_vali, verbose=1)
preds_val   = model.predict(x_new, verbose=1)
# preds_val   = model.predict(X_train[1726:1925], verbose=1)


# Threshold predictions
# preds_train_t = (preds_train > 0.5).astype(np.uint8)
# preds_val_t = (preds_val > 0.5).astype(np.uint8)

In [ ]:
def plot_sample(X, y, preds, ix=None):
    #squeeze() loss one dimension (128,128,1) (128,128)      #softmax use exp() to become[0,1]
    output = preds[ix].squeeze()  #     output = softmax(preds[ix].squeeze())*255
    new = []
    for i in range(128):
        new.append([])    
        for j in range(128):
            new[i].append(output[i][j])
            
            
    fig, ax = plt.subplots(1, 4, figsize=(20, 10))
    ax[0].imshow(X[ix, ..., 0],cmap='gray')
    ax[0].set_title('ILD')
    
    out_y = y[ix].squeeze()
    h = np.dstack([X[ix, ..., 0], X[ix, ..., 0], X[ix, ..., 0]])
    for i in range(128):
        for j in range(128):
            if out_y[i][j] != 0:
                h[i, j, :] = [1, 0, 0]
    

    ax[1].set_title('LABEL')
    ax[1].imshow(X[ix, ..., 0],cmap='gray')
    ax[1].imshow(h,alpha=0.5)

    

    ax[2].imshow(new)
    ax[2].set_title("direct predict")
    
    I = np.dstack([X[ix, ..., 0], X[ix, ..., 0], X[ix, ..., 0]])
    for i in range(128):
        for j in range(128):
            new[i][j] = sigmoid(new[i][j])*255
            if new[i][j] < 125 or new[i][j] > 129:
                new[i][j] = 255
                I[i, j, :] = [1, 0, 0]
            else:
                new[i][j] = 0
                
    ax[3].set_title('overlap')    
    ax[3].imshow(X[ix, ..., 0],cmap='gray')
    ax[3].imshow(I,alpha=0.5)
    

    fig.savefig('../../output_pic_unet2/'+str(ix)+'.png')
    

In [ ]:
def softmax(x):
    return scipy.special.softmax(x)
import math
def sigmoid(x):
#     print(x.shape)
    return 1 / (1 + math.exp(-x))

In [ ]:
for i in range(172):
    plot_sample(x_new, y_new, preds_val,ix=i)


In [ ]:
total = 0
count = 0
for i in range(172):
    white_sum = 0
    white_overlap = 0
    Dice = 0
     

    for j in range(128):
        for k in range(128):
            if y_new[i, j, k, 0]*255 >= 10:
                white_sum += 1
            if preds_val[i, j, k, 0]*255 > 10:
                white_sum += 1
            if y_new[i, j, k, 0]*255 >= 10 and preds_val[i, j, k, 0]*255 > 10:
                white_overlap += 1
    if white_sum != 0:
        Dice = (2*white_overlap) / white_sum
    else:
        Dice=0
    
    if(Dice!=0):
        total+=Dice
        count+=1
        print(Dice)
print()
print("average Dice:")
print(total/count)
print(count)